In [1]:
import pandas as pd
import imblearn
import numpy as np
import traceback
from dateutil.parser import parse
pd.set_option('display.max_rows', 20,'max_info_columns', 9999,'display.max_columns', 9999)
import matplotlib.pyplot as plt
import copy
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing,metrics
import datetime as dt
import gc
from sklearn import svm,linear_model
from collections import Counter
from sklearn.model_selection import train_test_split
import csv
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder,Imputer

from sklearn.ensemble import RandomForestRegressor,VotingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn import neighbors
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedKFold


from sklearn.feature_selection import RFE 
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
import xlrd
warnings.filterwarnings('ignore')

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('d_train_20180102.csv')
#train_=pd.read_csv('/home/tsl-yu/文档/天池-精准医疗/d_test_A_20180102.csv')
#answer=pd.read_csv('/home/tsl-yu/文档/天池-精准医疗/d_answer_a_20180128.csv',names='v')
test=pd.read_csv('d_test_B_20180128.csv')

mer=pd.read_excel('std.xlsx')

In [3]:
#train_['血糖']=answer

#train=pd.concat([train,train_],axis=0)

#train.reset_index(drop=True,inplace=True)

## 性别处理函数 

In [4]:
def sex(df):
    col=df.columns
    dic={ '男': 0 ,'女':1  }
    for c in col:
        if c=='性别':
            df[c]=df[c].map(dic)
            
    return df

## 年龄范围选择 

In [5]:
def select_age(df,n=10,m=100):
    df=df[df['年龄']>n][df['年龄']<m]
    return df

## label范围选择 

In [6]:
def select_label(df):
    
    ma=np.percentile(df['血糖'],98)
    df=df[df['血糖']<ma]
    return df

## 年龄阶段划分函数 

In [7]:
def age_split(df,n=5):
    col=df.columns
    
    for c in col:
        if c=='年龄':
            df['age_cata']=pd.cut(df[c],n,labels=False)
            
    return df

##  空值填充|

In [8]:
def nul_fill(df):
    for col in df.columns:
        l=df[col][df[col].isnull()]
        if len(l)!=len(df[col]):
            mean=np.nanmean(df[col])
            df[col]=df[col].fillna(mean)
        
    return df

## 日期处理函数 

In [9]:
def pro_date(df):
    col=df.columns
    
    for c in col:
        if c=='体检日期':
            df['体检日期']=[dt.datetime.strptime(str,'%d/%m/%Y') for str in df['体检日期']]
            
            df['month']=[date.month for date in df['体检日期']]
            df['day']=[date.day for date in df['体检日期']]
            df['weekday']=[date.dayofweek for date in df['体检日期']]
            df['体检日期'] = (pd.to_datetime(df['体检日期']) - parse('2017-10-09')).dt.days
            
    return df

##  去掉空值超过0.7的函数

In [10]:
def drop_null(df):
    for col in df.columns:
        a=df[col].isnull().sum()
        if a>len(df[col])*0.7:
            df=df.drop(col,axis=1)
            
    return df

##  以年龄、性别为依据，分组填充

In [11]:
def fill_nul(df,n=27):
    col=df.columns    
    for c in col:
        if c=='年龄':
            df['split']=pd.cut(df[c],n,labels=False)
         
    for age,group in df.groupby('split'):
            for sex,data in group.groupby('性别'):
                index=data.index
                df.iloc[index]=nul_fill(data)
    df=df.drop('split',axis=1)
    return df

## 对是否是高血糖 进行判断产生label

In [12]:
def b_g(label,x=10):
    out=[]
    for a in label:

        if a>x:
            out.append(1)
        else:
            out.append(0)
    return np.array(out)
    

正常血糖范围为3.9-6.1

## 利用标准数据产生新特征 

In [13]:
def new_feature(data,std):
    df=copy.deepcopy(data)
    col_df=df.columns
    col_std=std.columns
    for i,col in enumerate(col_df):
        if col in col_std:
            df[str(i)+'std']=df[col]-std[col][0]
            df[str(i)+'_std']=df[str(i)+'std']/std[col][0]
            index=df[df['性别']==1].index
            df[str(i)+'std'][index]=df[col][index]-std[col][1]
            df[str(i)+'_std'][index]=df[str(i)+'std'][index]/std[col][1]
    return df

##  利用四则运算构建特征

In [14]:
def base_alg(data):
    df=copy.deepcopy(data)
    col=df.columns
    for i in range(len(col)):
        for j in range(len(col)-i-1):
            df[col[i]+'_add_'+col[i+1+j]]=df[col[i]]+df[col[i+1+j]]
            df[col[i]+'_sub_'+col[i+1+j]]=df[col[i]]-df[col[i+1+j]]
            df[col[i]+'_mult_'+col[i+1+j]]=df[col[i]]*df[col[i+1+j]]
            
            a=[]
            for k in range(len(df[col[i]])):
                if np.fabs(df[col[i+1+j]][k])>0.0001:
                              a.append(df[col[i]][k]/df[col[i+1+j]][k])
                else:
                              a.append(None)
            df[col[i]+'_div_'+col[i+1+j]]=a
    return df

## 交叉验证函数 

In [15]:
def cv_function(model,train,label,test,n=10):
    try:
        kf = KFold(n_splits=n,shuffle=True)
        l=[]
        
        err=[]
        if type(train)==pd.DataFrame:
            train=train.fillna(-999).values
        if type(test)==pd.DataFrame:
            test=test.fillna(-999).values
        
        if type(label)==pd.core.series.Series:
            label=label.values
    
        for tr_index,te in kf.split(train,label):
            train_x,test_x=train[tr_index],train[te]
            train_y,test_y=label[tr_index],label[te]
            model.fit(train_x,train_y)
            a=model.predict(test_x)
            err.append(np.mean((a-test_y)**2))
            print (err[-1])
            
            out=model.predict(test)
            l.append(out)
                
            
        print('mean error   ',np.mean(err))
        return l,err
    except :
        print('error')

##  交叉验证，分类评测结果

In [16]:
def cla_cv_function(model,train,label,test,n=10):
    try:
        kf = KFold(n_splits=n,shuffle=True)
        l=[]
        
        err=[]
        if type(train)==pd.DataFrame:
            train=train.fillna(-999).values
        if type(test)==pd.DataFrame:
            test=test.fillna(-999).values
        
        if type(label)==pd.core.series.Series:
            label=label.values
    
        for tr_index,te in kf.split(train,label):
            train_x,test_x=train[tr_index],train[te]
            train_y,test_y=label[tr_index],label[te]
            model.fit(train_x,train_y)
            a=model.predict(test_x)
            err.append(metrics.f1_score(test_y,a))
            print (metrics.confusion_matrix(test_y,a))
            
            out=model.predict(test)
            l.append(out)
                
            
        print('mean error   ',np.mean(err))
        return l,err
    except :
        print('error')

## 多项式特征 

In [17]:
def poly_f(df,n=3):
    py=PolynomialFeatures(degree=n,interaction_only=True)
    return py.fit_transform(df) 

##  训练数据构建

In [18]:
train=select_age(train)
#train=select_label(train)

In [19]:
#train=train[train['血糖']<30]

In [20]:
label=train['血糖']

train1=pd.concat([train.drop('血糖',axis=1),test],axis=0,ignore_index=True)
train1.drop('id',axis=1,inplace=True)

In [21]:
train1=sex(train1)
#train1=age_split(train1,8)
train1=pro_date(train1)
train1=drop_null(train1)

In [22]:
train1=fill_nul(train1)
train1=fill_nul(train1,20)

In [23]:
train1=train1.fillna(train1.median(axis=0))

In [24]:
train1.shape

(6640, 38)

In [25]:
ss=StandardScaler()

In [26]:
train2=ss.fit_transform(train1)

In [27]:
train2=pd.DataFrame(train2,columns=train1.columns)

In [28]:
b1=base_alg(train1)
b2=base_alg(train2)

In [29]:
b1=b1.fillna(b1.median(axis=0))
b2=b2.fillna(b2.median(axis=0))

In [ ]:
test.info()

### 多项式数据 

In [ ]:
pp=poly_f(train2,3)

In [ ]:
pp=pd.DataFrame(pp)

In [ ]:
pp.columns=[str(i)+'_s' for i in range(len(pp.columns))]

## 数据分析 

In [ ]:
plt.figure(figsize=(5,5))
sns.distplot(train['血糖'])

 实际分析发现血糖分布：
   大于6.1             911
   大于10              139
   大于15              27
   大于20              4

In [ ]:
np.percentile(train['血糖'],95)

In [ ]:
label.shape

## 对不均匀数据进行采样 

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from imblearn.ensemble import EasyEnsemble

In [ ]:
smote_enn =SMOTEENN(random_state=0,n_neighbors=6,n_jobs=-1,m=3,ratio=0.5)

In [ ]:
x_sam,y_sam=smote_enn.fit_sample(train2[:5642,:],b_g(label,10))

In [ ]:
sns.countplot(y_sam)

In [ ]:
test_y[test_y>15]

In [ ]:
ind_a,ind_b=list(label[label>7].index),list(label[label<7].index)

In [ ]:
sp_train2,hp_train2=train2.iloc[ind_a],train2.iloc[ind_b]
sp_label,hp_label=label[ind_a],label[ind_b]

##  模型融合

In [ ]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
        
        from sklearn.cross_validation import KFold
        kf=KFold(self.n_folds)
        
    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        
        
        kf=KFold(self.n_folds)
        folds=kf.split(X)
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0],self.n_folds))
            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]
            S_test[:, i] = S_test_i.mean(1)
        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred

In [ ]:
train1

##  XGB 模型

In [94]:
train_x,test_x,train_y,test_y=train_test_split(b2.iloc[:5640],label,test_size=0.1,random_state=0)

train_=xgb.DMatrix(train_x,train_y)
test_=xgb.DMatrix(test_x,test_y)

watchlist=[(train_,'train'),(test_,'test')]

In [167]:
    params = {
            'booster' : 'gbtree',
            'objective': 'reg:linear',
            'eta': 0.06,
            'gamma' : 0.01,
            'max_depth' :4,
            'min_child_weight' :1,
            'eval_metric':'rmse',
            'seed': 11,
            'missing': -999,
            'subsample' :0.8,
        
            'colsample_bytree' :0.11,
             'colsample_bybooster':1,
          
            'silent' : 1,
            
            
            }

In [168]:
model = xgb.train(params,train_,num_boost_round=500 ,evals=watchlist,early_stopping_rounds=30)

[0]	train-rmse:5.06211	test-rmse:5.04924
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 30 rounds.
[1]	train-rmse:4.78211	test-rmse:4.77159
[2]	train-rmse:4.52154	test-rmse:4.51485
[3]	train-rmse:4.27628	test-rmse:4.27493
[4]	train-rmse:4.04833	test-rmse:4.05197
[5]	train-rmse:3.83373	test-rmse:3.83831
[6]	train-rmse:3.63196	test-rmse:3.64307
[7]	train-rmse:3.44423	test-rmse:3.45831
[8]	train-rmse:3.26866	test-rmse:3.28415
[9]	train-rmse:3.1064	test-rmse:3.1247
[10]	train-rmse:2.95278	test-rmse:2.97636
[11]	train-rmse:2.80957	test-rmse:2.83944
[12]	train-rmse:2.67762	test-rmse:2.71143
[13]	train-rmse:2.55339	test-rmse:2.59317
[14]	train-rmse:2.43896	test-rmse:2.48424
[15]	train-rmse:2.33127	test-rmse:2.38285
[16]	train-rmse:2.23109	test-rmse:2.28507
[17]	train-rmse:2.14131	test-rmse:2.19768
[18]	train-rmse:2.05654	test-rmse:2.11939
[19]	train-rmse:1.97715	test-rmse:2.04758
[20]	train-rmse:1.90231	test-

In [169]:
score=model.get_score()

impt=pd.DataFrame()
impt['name']=score.keys()
impt['importance']=score.values()

impt=impt.sort_values(by ='importance',axis=0,ascending=False)
impt.reset_index(drop=True,inplace=True)

In [170]:
a=impt[impt['importance']>1]['name']

In [171]:
a

0                 *r-谷氨酰基转换酶
1         性别_add_*天门冬氨酸氨基转换酶
2                  *丙氨酸氨基转换酶
3                        白蛋白
4                         年龄
5          年龄_add_*r-谷氨酰基转换酶
6           年龄_sub_红细胞体积分布宽度
7                       体检日期
8                     *碱性磷酸酶
9           年龄_sub_高密度脂蛋白胆固醇
               ...          
349             年龄_add_红细胞压积
350           *球蛋白_add_红细胞计数
351          白细胞计数_div_单核细胞%
352       白蛋白_mult_红细胞体积分布宽度
353      总胆固醇_mult_血小板体积分布宽度
354              年龄_sub_*总蛋白
355     红细胞平均体积_mult_血小板平均体积
356       *碱性磷酸酶_div_红细胞平均体积
357    *天门冬氨酸氨基转换酶_div_红细胞计数
358          白蛋白_sub_红细胞平均体积
Name: name, Length: 359, dtype: object

In [ ]:
out1=model.predict(xgb.DMatrix(train2.iloc[5642:,:]))

In [ ]:
sns.jointplot(x=out,y=out1,kind='reg')

In [ ]:
np.mean((np.exp(out)-np.exp(test_y))**2)

In [ ]:
pd.DataFrame(out1).to_csv('random_4.csv',index=False,header=False)

In [ ]:
train1.shape

In [178]:
xrg=xgb.XGBRegressor(max_depth=4,learning_rate=0.06,
n_estimators=121,nthread=-1,gamma=1,colsample_bytree=0.11,subsample=0.8,
missing=-999,colsample_bylevel=1)

In [180]:
k,err=cv_function(xrg,b2[a].iloc[:5640],label,b2[a].iloc[5640:],n=10)

2.28606409337
1.88092634094
1.70505539537
1.552281221
1.13584243149
1.72162675516
3.72929427054
1.31501630128
1.76340310461
1.79870295291
mean error    1.88882128667


In [181]:
err1=[1/a for a in err]
err1=err1/sum(err1)

out_xgb=[(k[i])*err1[i] for i in range(len(k))]

out_xgb=sum(k)/len(k)

In [182]:
pd.DataFrame(out_xgb).to_csv('xgb_b.csv',index=False,header=False)

##  循环特征选择

###  Pearson Correlation

In [198]:
from scipy.stats import spearmanr

In [199]:
col=b2.columns

In [200]:
sp_=[]

for c in col:
    sp_.append(spearmanr(b2[c][:5640],label)[0])

In [201]:
corr=pd.DataFrame()
corr['name']=col
corr['valu']=sp_
corr=corr.sort_values(by ='valu',axis=0,ascending=False)
corr.reset_index(drop=True,inplace=True)

### 最大互信息商 

In [ ]:
from minepy import MINE

In [ ]:
mifm = MINE() 

In [ ]:
sp_1=[]

for c in col:
    mifm.compute_score(pp[c][:5642],label) 
    sp_1.append(mifm.mic())

In [ ]:
corr1=pd.DataFrame()
corr1['name']=col
corr1['valu']=sp_1
corr1=corr1.sort_values(by ='valu',axis=0,ascending=False)
corr1.reset_index(drop=True,inplace=True)

In [ ]:
corr1

### 递归消除特征 

In [ ]:
sel = RFE(reg, 1, step=1)

In [ ]:
sel.fit(pp.iloc[:5642],label)

In [ ]:
corr2=pd.DataFrame()
corr2['name']=col
corr2['valu']=sel.ranking_
corr2=corr2.sort_values(by ='valu',axis=0,ascending=False)
corr2.reset_index(drop=True,inplace=True)

In [ ]:
corr2

In [ ]:
corr2

## Lgb 模型 

In [184]:
train_x1,test_x1,train_y1,test_y1=train_test_split(b2[a].iloc[:5640],label,test_size=0.1,random_state=0)

train_1=lgb.Dataset(train_x1,label=train_y1)#,feature_name=list(train_x1.columns))
test_1=lgb.Dataset(test_x1,label=test_y1,)#feature_name=list(test_x1.columns))

In [185]:
param = {
        'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'sub_feature': 0.9,
    'num_leaves': 16,
    'colsample_bytree': 0.8,

    'max_depth':4,


    'min_split_gain':1,
    'scale_pos_weight':1,
    'num_threads':-1,
    
    'bagging_fraction' :0.81,
    'feature_fraction':0.51,
    'metric':'rmse',
    'subsample':0.8

}
param['is_unbalance']='true'

In [186]:
model=lgb.train(param,train_1,num_boost_round=1500,valid_sets=[train_1,test_1],early_stopping_rounds=25)

[1]	training's rmse: 1.54826	valid_1's rmse: 1.48086
Training until validation scores don't improve for 25 rounds.
[2]	training's rmse: 1.54546	valid_1's rmse: 1.47941
[3]	training's rmse: 1.5427	valid_1's rmse: 1.47799
[4]	training's rmse: 1.53975	valid_1's rmse: 1.4769
[5]	training's rmse: 1.5372	valid_1's rmse: 1.47584
[6]	training's rmse: 1.5343	valid_1's rmse: 1.47453
[7]	training's rmse: 1.53196	valid_1's rmse: 1.47305
[8]	training's rmse: 1.52961	valid_1's rmse: 1.47183
[9]	training's rmse: 1.52719	valid_1's rmse: 1.47087
[10]	training's rmse: 1.52418	valid_1's rmse: 1.46886
[11]	training's rmse: 1.52139	valid_1's rmse: 1.46771
[12]	training's rmse: 1.5187	valid_1's rmse: 1.46688
[13]	training's rmse: 1.51566	valid_1's rmse: 1.46567
[14]	training's rmse: 1.51298	valid_1's rmse: 1.4649
[15]	training's rmse: 1.51049	valid_1's rmse: 1.46352
[16]	training's rmse: 1.50765	valid_1's rmse: 1.46218
[17]	training's rmse: 1.50508	valid_1's rmse: 1.46124
[18]	training's rmse: 1.50247	valid

[230]	training's rmse: 1.21023	valid_1's rmse: 1.37752
[231]	training's rmse: 1.20963	valid_1's rmse: 1.37731
[232]	training's rmse: 1.20918	valid_1's rmse: 1.37733
[233]	training's rmse: 1.20839	valid_1's rmse: 1.37745
[234]	training's rmse: 1.20775	valid_1's rmse: 1.37745
[235]	training's rmse: 1.20662	valid_1's rmse: 1.3769
[236]	training's rmse: 1.20567	valid_1's rmse: 1.37698
[237]	training's rmse: 1.20527	valid_1's rmse: 1.37675
[238]	training's rmse: 1.20453	valid_1's rmse: 1.37691
[239]	training's rmse: 1.204	valid_1's rmse: 1.37697
[240]	training's rmse: 1.2036	valid_1's rmse: 1.37695
[241]	training's rmse: 1.20247	valid_1's rmse: 1.37533
[242]	training's rmse: 1.20204	valid_1's rmse: 1.3753
[243]	training's rmse: 1.20138	valid_1's rmse: 1.37528
[244]	training's rmse: 1.20049	valid_1's rmse: 1.37535
[245]	training's rmse: 1.19987	valid_1's rmse: 1.37545
[246]	training's rmse: 1.1992	valid_1's rmse: 1.37561
[247]	training's rmse: 1.19866	valid_1's rmse: 1.37547
[248]	training's

[400]	training's rmse: 1.1068	valid_1's rmse: 1.36838
Early stopping, best iteration is:
[375]	training's rmse: 1.11904	valid_1's rmse: 1.36672


In [195]:
lrg=lgb.LGBMRegressor( colsample_bytree=0.11, learning_rate=0.01,
       max_bin=255, max_depth=4, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.51, n_estimators=400,
       n_jobs=-1, num_leaves=16, objective=None, random_state=None,
       reg_alpha=0.1, reg_lambda=0.0, silent=True, subsample=0.8,
       subsample_for_bin=200000, subsample_freq=1)

In [196]:
lk,lerr=cv_function(lrg,b2[a].iloc[:5640],label,b2[a].iloc[5640:],n=10)

1.70761337411
1.6792326855
1.21714229079
1.79440098592
1.98860230649
2.01986148381
1.47491435085
1.9177979796
1.64041294175
3.49843020117
mean error    1.89384086


In [197]:
lerr1=[1/a for a in lerr]
lerr1=lerr1/sum(lerr1)

out_lgb=[lk[i]*lerr1[i] for i in range(len(lk))]

out_lgb=sum(lk)/len(lk)

In [ ]:
pd.DataFrame(linear_ensembel).to_csv('linear_ensembel_b.csv',index=False,header=False)

## xgb 产生新特征 

In [ ]:
gbdt=xgb.XGBRegressor(max_depth=4,learning_rate=0.06,
n_estimators=180,nthread=-1,gamma=0.0,colsample_bytree=1,
missing=-999,colsample_bylevel=1)

In [ ]:
gbdt.fit(train1.iloc[:5642].fillna(-999),label)

In [ ]:
tre=pd.DataFrame(gbdt.apply(train1.fillna(-999)))

## 支持向量机模型 

In [ ]:
svc= svm.SVC(probability=True)

In [ ]:
svc.fit(x_sam,y_sam)

In [ ]:
prob2=svc.predict(train2[:5642])

In [ ]:
metrics.accuracy_score(b_g(label,10),prob2)

In [ ]:
metrics.recall_score(b_g(label,10),prob2)

In [ ]:
metrics.f1_score(b_g(label,10),prob2)

In [ ]:
metrics.confusion_matrix(b_g(label,10),prob2)

In [ ]:
Counter(prob2)

In [ ]:
Counter(b_g(label,10))

## voting

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
svc= svm.SVC(probability=True)
knn=neighbors.KNeighborsClassifier()
xgbc=xgb.XGBClassifier(max_depth=6,learning_rate=0.1,
n_estimators=261,nthread=-1,gamma=0.0,colsample_bytree=1,
missing=-999,colsample_bylevel=1)
sgd=SGDClassifier()

In [ ]:
voting_clf = VotingClassifier( estimators=[("svc",svc),('knn',knn),('xgbc',xgbc),('sgd',sgd)])

In [ ]:
voting_clf.fit(x_sam,y_sam)

In [ ]:
prob2=voting_clf.predict(train2[:5642])

## KNN  算法 

In [ ]:
knn=neighbors.KNeighborsClassifier()

In [ ]:
knn.fit(x_sam,y_sam)

In [ ]:
out_knn=knn.predict(train2[:5642])

In [ ]:
Counter(out_knn)

## 线性模型 

In [207]:
reg = linear_model.RidgeCV(alphas=[0.1,50.0,100])

In [208]:
reg.fit(b2[corr[abs(corr['valu'])>0.132]['name']].iloc[:5640],label)

RidgeCV(alphas=[0.1, 50.0, 100], cv=None, fit_intercept=True, gcv_mode=None,
    normalize=False, scoring=None, store_cv_values=False)

In [209]:
reg.alpha_

100.0

In [210]:
k,err=cv_function(reg,b2[corr[abs(corr['valu'])>0.145]['name']].iloc[:5640],label,b2[corr[abs(corr['valu'])>0.145]['name']].iloc[5640:],n=10)

1.27112560055
2.42905177195
4.35054929952
1.34867010643
1.42234158369
2.22859646527
2.28637701621
1.80281829261
1.39009051055
1.41711746402
mean error    1.99467381108


In [226]:
err1=[1/a for a in err]
err1=err1/sum(err1)

out_ridg=[(k[i])*err1[i] for i in range(len(k))]

out_ridg=sum(k)/len(k)

In [229]:
pd.DataFrame(out_fina).to_csv('ensem_linear_lgb_xgb_b_wu.csv',index=False,header=False)

### elastic net


In [214]:
regr=linear_model.ElasticNetCV(cv=5,l1_ratio=[.01,.1, .5, 1])

In [215]:

regr.fit(b2[corr[abs(corr['valu'])>0.132]['name']].iloc[:5640,:],label)

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
       l1_ratio=[0.01, 0.1, 0.5, 1], max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0)

In [216]:
regr.alpha_

0.0053464163406754441

In [217]:
k,err=cv_function(regr,b2[corr[abs(corr['valu'])>0.132]['name']].iloc[:5640,:],label,b2[corr[abs(corr['valu'])>0.132]['name']].iloc[5640:],n=10)

4.01450657814
1.50598899301
1.71941559419
1.51929952177
1.85361843225
1.52433268845
2.31264866223
1.48283260762
2.00527579064
1.84348878522
mean error    1.97814076535


In [218]:
err1=[1/a for a in err]
err1=err1/sum(err1)

out_ela=[(k[i])*err1[i] for i in range(len(k))]

out_ela=sum(k)/len(k)

## kernel ridge 

In [ ]:
krr=KernelRidge(alpha=100)

In [ ]:
k,err=cv_function(krr,train2[:5641,:],np.log1p(label),train2[5641:,:],n=10)

## lasso 

In [220]:
lasso=linear_model.LassoCV(alphas=[0.01,0.1,1,10])

In [222]:
lasso.fit(b2[corr[abs(corr['valu'])>0.15]['name']].iloc[:5640,:],label)

LassoCV(alphas=[0.01, 0.1, 1, 10], copy_X=True, cv=None, eps=0.001,
    fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=1,
    normalize=False, positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [ ]:
lasso.alpha_

In [223]:
k,err=cv_function(lasso,b2[corr[abs(corr['valu'])>0.15]['name']].iloc[:5640,:],label,b2[corr[abs(corr['valu'])>0.15]['name']].iloc[5640:],n=10)

1.38922333277
2.42022998209
1.74726305008
3.84936983366
2.25527802202
1.86840441812
1.69010845968
1.57548155486
1.01758204032
2.04946073946
mean error    1.98624014331


In [224]:
err1=[1/a for a in err]
err1=err1/sum(err1)

out_las=[(k[i])*err1[i] for i in range(len(k))]

out_las=sum(k)/len(k)

In [ ]:
train1[impt[impt['importance']>2]['name']].iloc[:5642].shape

In [ ]:
label.shape

In [ ]:
train1.fillna(0,inplace=True)

In [ ]:
train1.isnull().sum().sum()

In [ ]:
np.max(train1.values)

In [ ]:
pd.concat([impt,corr,corr1],axis=1)

In [ ]:
corr1[corr1['valu']>0.089]['name']

In [ ]:
out=(out_ela+out_las+out_ridg)/3

In [227]:
linear_ensembel=(out_ela+out_las+out_ridg)/3

In [228]:
out_fina=(linear_ensembel+out_lgb+out_xgb)/3

In [ ]:
out_ridg